In [2]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.time import Time
from astroquery.mast import Mast,Observations
from astropy.table import Table
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import json

In [3]:
def set_params(parameters):
    return [{"paramName" : p, "values" : v} for p, v in parameters.items()]

def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }


In [5]:
service = 'Mast.Jwst.Filtered.Miri'

keywords = {
            'exp_type':['MIR_CORONCAL'],
            'instrume':['MIRI'],
            'date_obs_mjd': [set_mjd_range('2022-01-01','2022-10-27')],
           }

params = {'columns': '*',
          'filters': set_params(keywords)
         }



In [6]:
t = Mast.service_request(service, params)
display_columns = [x for x in t.colnames]

In [7]:
t[display_columns]

ArchiveFileID,filename,fileSetName,productLevel,act_id,apername,asnpool,asntable,bartdelt,bendtime,bkgdtarg,bkglevel,bkgsub,bmidtime,bstrtime,category,cont_id,datamode,dataprob,date,date_mjd,date_end,date_end_mjd,date_obs,date_obs_mjd,detector,drpfrms1,drpfrms3,duration,effexptm,effinttm,eng_qual,exp_type,expcount,expend,expmid,exposure,expripar,expstart,fastaxis,filter,frmdivsr,gainfact,gdstarid,groupgap,gs_dec,gs_mag,gs_order,gs_ra,gsendtim,gsendtim_mjd,gsstrttm,gsstrttm_mjd,gs_udec,gs_umag,gs_ura,helidelt,hendtime,hga_move,hmidtime,hstrtime,instrume,intarget,is_psf,lamp,mu_dec,mu_epoch,mu_epoch_mjd,mu_ra,nexposur,nextend,nframes,ngroups,nints,nresets,nrststrt,nsamples,numdthpt,nwfsest,obs_id,observtn,obslabel,origin,pcs_mode,pi_name,pps_aper,prd_ver,program,prop_dec,prop_ra,pwfseet,readpatt,sca_num,scicat,sdp_ver,selfref,seq_id,slowaxis,subarray,subcat,subsize1,subsize2,substrt1,substrt2,targ_dec,targ_ra,targname,targoopp,targprop,targtype,targudec,targura,telescop,template,tframe,tgroup,timesys,title,tsample,tsovisit,visit,visit_id,visitend,visitend_mjd,visitgrp,visitsta,visitype,vststart,vststart_mjd,xoffset,yoffset,zerofram,errtype,rois,roiw,wpower,wtype,datamodl,exp_only,exsegnum,exsegtot,intstart,intend,date_beg,date_beg_mjd,obsfoldr,sctarate,opmode,osf_file,expsteng,expsteng_mjd,masterbg,scatfile,srctyapt,tcatfile,texptime,patt_num,pattsize,patttype,pridtpts,subpxpts,crowdfld,engqlptg,oss_ver,noutputs,gs_v3_pa,dirimage,pixfrac,pxsclrt,segmfile,va_dec,va_ra,compress,bkgmeth,s_region,cal_ver,cal_vcs,crds_ctx,crds_ver,band,cccstate,channel,coronmsk,pattstrt,spat_num,spatnstp,spatstep,spcoffst,spec_num,specnstp,specstep,sptoffst,mirngrps,mirnfrms,dithdirc,dithopfr,dithpnts,dsetstrt,mrsprchn,numdsets,pattnpts,detmode,cmd_tsel,fpe_side,ice_side,fileSize,checksum,ingestStartDate,ingestStartDate_mjd,ingestCompletionDate,ingestCompletionDate_mjd,FileTypeID,publicReleaseDate,publicReleaseDate_mjd,isRestricted,isItar,isStale,FileSetId,dataURI
str8,str48,str25,str2,str2,str14,str32,str48,float64,float64,str1,float64,str1,float64,float64,str3,int64,int64,str1,str21,float64,str21,float64,str21,float64,str8,int64,int64,float64,float64,float64,str2,str12,int64,float64,float64,int64,str5,float64,int64,str6,int64,float64,str10,int64,float64,float64,int64,float64,str21,float64,str21,float64,float64,float64,float64,float64,float64,str1,float64,float64,str4,str1,str1,str3,float64,str21,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,str26,int64,str17,str5,str9,str15,str14,str13,int64,float64,float64,float64,str6,int64,str1,str7,str1,str1,int64,str8,str5,int64,int64,int64,int64,float64,float64,str14,str1,str20,str5,float64,float64,str4,str42,float64,float64,str3,str80,float64,str1,int64,str11,str21,float64,str2,str12,str20,str21,float64,float64,float64,str1,str1,float64,float64,float64,str1,str10,str1,int64,int64,int64,int64,str21,float64,str20,float64,str1,str29,str1,float64,str1,str1,str1,str1,float64,int64,str1,str12,int64,int64,str1,str26,str15,int64,float64,str1,float64,float64,str1,float64,float64,str1,str1,str125,str5,str7,str14,str8,str1,str4,str1,str9,int64,int64,int64,float64,float64,int64,int64,float64,float64,int64,int64,str1,str1,str1,int64,str1,int64,int64,str1,str1,str1,str1,str7,str32,str21,float64,str21,float64,int64,str21,float64,bool,bool,bool,str7,str66
5948650,jw01045002001_02103_00004_mirimage_cal.fits,jw01045002001_02103_00004,2b,03,MIRIM_MASK1140,jw01045_20220703t030200_pool.csv,jw01045-o002_20220703t030200_image2_003_asn.json,-13.64389979280531,59732.32809730683,f,nan,--,59732.32805985562,59732.32802240441,COM,--,7,f,/Date(1656818835124)/,59763.143925034725,/Date(1654156361251)/,59732.32825520833,/Date(1654156354780)/,59732.328180324075,MIRIMAGE,0,0,6.471,6.471,6.47136,OK,MIR_CORONCAL,5,59732.32825521991,59732.32821776991,4,PRIME,59732.32818031991,1,F1140C,1,nan,N4FF006082,0,60.52518507196048,16.10432624816895,1,261.0154570613097,/Date(1654155570467)/,59732.319102615744,/Date(165415548

In [8]:
fn = list(set(t['filename']))

ids = list(set(['_'.join(x.split('_')[:-1]) for x in fn]))

In [9]:
matched_obs = Observations.query_criteria(instrument_name='Miri',
                                          obs_id=ids,
                                         )

In [10]:
display_cols = [x for x in matched_obs.columns]
matched_obs[display_cols]

dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
str5,int64,str4,str34,str14,float64,float64,float64,float64,float64,str8,str6,float64,float64,str1,str80,float64,str4,str15,str4,str4,str4,int64,str7,str119,str61,str61,str6,bool,float64,str7,str8,str9
image,2,JWST,jw01532001001_02101_00001_mirimage,Zodiacal light,229.9390733333333,-11.70366388888891,59778.25003800926,59778.25045412037,35.952,Infrared,F1065C,10385.0,10915.0,--,Coronagraph External Flatfield,59780.86562492,MIRI,"Hines, Dean C.",1532,JWST,JWST,--,CALJWST,POLYGON 229.937125652 -11.695566558 229.943526056 -11.698213442 229.940051512 -11.706353608 229.933697503 -11.7037121,mast:JWST/product/jw01532001001_02101_00001_mirimage_rate.jpg,mast:JWST/product/jw01532001001_02101_00001_mirimage_cal.fits,PUBLIC,False,nan,science,87632159,167130572
image,2,JWST,jw01532003001_02101_00001_mirimage,Zodiacal light,229.9390733333333,-11.70366388888891,59778.26476021991,59778.26517633102,35.952,Infrared,F1550C,15110.0,15889.999999999998,--,Coronagraph External Flatfield,59780.86662024,MIRI,"Hines, Dean C.",1532,JWST,JWST,--,CALJWST,POLYGON 229.937070321 -11.695578142 229.943585606 -11.698243446 229.94006625 -11.706339126 229.933566866 -11.703699547,mast:JWST/product/jw01532003001_02101_00001_mirimage_rate.jpg,mast:JWST/product/jw01532003001_02101_00001_mirimage_cal.fits,PUBLIC,False,nan,science,87632149,167130744
image,2,JWST,jw01532003001_02101_00002_mirimage,Zodiacal light,229.9390733333333,-11.70366388888891,59778.26613355324,59778.26654966435,35.952,Infrared,F1550C,15110.0,15889.999999999998,--,Coronagraph External Flatfield,59780.86783553,MIRI,"Hines, Dean C.",1532,JWST,JWST,--,CALJWST,POLYGON 229.936019524 -11.694874389 229.942534779 -11.697539724 229.939015391 -11.705635388 229.932516037 -11.702995777,mast:JWST/product/jw01532003001_02101_00002_mirimage_rate.jpg,mast:JWST/product/jw01532003001_02101_00002_mirimage_cal.fits,PUBLIC,False,nan,science,87632154,167130902
image,2,JWST,jw01532003001_02101_00003_mirimage,Zodiacal light,229.9390733333333,-11.70366388888891,59778.26747354167,59778.26788965278,35.952,Infrared,F1550C,15110.0,15889.999999999998,--,Coronagraph External Flatfield,59780.86638876,MIRI,"Hines, Dean C.",1532,JWST,JWST,--,CALJWST,POLYGON 229.937158818 -11.69436507 229.943674072 -11.697030379 229.940154725 -11.705126057 229.933655372 -11.702486472,mast:JWST/product/jw01532003001_02101_00003_mirimage_rate.jpg,mast:JWST/product/jw01532003001_02101_00003_mirimage_cal.fits,PUBLIC,False,nan,science,87632152,167130837
image,2,JWST,jw01532003001_02101_00004_mirimage,Zodiacal light,229.9390733333333,-11.70366388888891,59778.268846875,59778.26926298611,35.952,Infrared,F1550C,15110.0,15889.999999999998,--,Coronagraph External Flatfield,59780.86600682,MIRI,"Hines, Dean C.",1532,JWST,JWST,--,CALJWST,POLYGON 229.936579134 -11.694723787 229.94309439 -11.697389112 229.939575018 -11.705484781 229.933075663 -11.702845182,mast:JWST/product/jw01532003001_02101_00004_mirimage_rate.jpg,mast:JWST/product/jw01532003001_02101_00004_mirimage_cal.fits,PUBLIC,False,nan,science,87632155,167130922
image,2,JWST,jw01532003001_02103_00001_mirimage,Zodiacal light,229.9390733333333,-11.70366388888891,59778.27093280092,59778.27134891204,35.952,Infrared,F1550C,15110.0,15889.999999999998,--,Coronagraph External Flatfield,59780.86586797,MIRI,"Hines, Dean C.",1532,JWST,JWST,--,CALJWST,POLYGON 229.93537699 -11.699587926 229.941892348 -11.70225328 229.938372876 -11.710348933 229.931873419 -11.707709305,mast:JWST/product/jw01532003001_02103_00001_mirimage_rate.jpg,mast:JWST/product/jw01532003001_02103_00001_mirimage_cal.fits,PUBLIC,False,nan,science,87632157,167130981
...,...,...,...,...,...,...,...,...,...,...

In [11]:
print(f'Found {len(matched_obs)} matching Observations.')

Found 319 matching Observations.


In [12]:
matched_obs[display_cols]
batch_size = 5
batches = [matched_obs[i:i+batch_size] for i in range(0, len(matched_obs), batch_size)]
t = [Observations.get_product_list(obs) for obs in batches]
products = unique(vstack(t), keys='productFilename')
print(f'  Number of unique products: {len(products)}')

  Number of unique products: 4685


In [ ]:
filtered_products = Observations.filter_products(products,
                                                 #dataproduct_type='image',
                                                 #productType = ['SCIENCE'],
                                                 #proposal_id = ['1075'],
                                                 productSubGroupDescription = ['RATEINTS',
                                                                               #'PSFSTACK',
                                                                               #'CALINTS',
                                                                               #'CRFINTS',
                                                                               #'PSFALIGN',
                                                                               'POOL',
                                                                               'ASN'])